In [23]:
# Import the required module from the fyers_apiv3 package
from fyers_apiv3 import fyersModel
from fyers_apiv3.FyersWebsocket import data_ws
import credentials as cd
from Mongo import MongoAPI
import pandas as pd
import datetime as dt
import time
import numpy as np
import math
from datetime import date
mongoObj= MongoAPI('logs')

with open('access.txt','r') as a:
    access_token=a.read()
client_id = cd.client_id
#variables
pos = 0
stoploss = 0
entry = 0
flag = 0
exit = 0
target = 0
trike = ''
row = -2
expiry = 'NSE:BANKNIFTY24117'
sym = 'NSE:NIFTYBANK-INDEX'
gain = 0
fmflag = 0
fimflag = 0
emadata5 = pd.DataFrame()
emadata15 = pd.DataFrame()
date_from = '2024-11-21'
date_to = '2024-11-22'

#GET DATA FUNCTION 
def getdata():
    cdata = {
            "symbol":sym,
            "resolution":str(res),
            "date_format":"1",
            "range_from":rfrom,
            "range_to":rto,
            "cont_flag":"0"
            }
    response = fyers.history(data=cdata)
    data = pd.DataFrame.from_dict(response['candles'])
    cols = ['datetime', 'open', 'high','low','close', 'volume']
    data.columns = cols
    data['datetime'] = pd.to_datetime(data['datetime'], unit="s")
    data['datetime'] = data['datetime'].dt.tz_localize('utc').dt.tz_convert('Asia/Kolkata')
    data['datetime'] = data['datetime'].dt.tz_localize(None)
    data = data.set_index('datetime')
    data["ema"] = data["close"].ewm(span = 5 , min_periods=5).mean()
    global emadata
    emadata = data


<---------------------------------->
|       DB initialized             |
<---------------------------------->


In [10]:
getdata()
emadata

,open,high,low,close,volume,ema
datetime,,,,,,
2024-11-21 09:15:00,50625.00,50652.15,50140.75,50190.00,0,NaN
2024-11-21 09:20:00,50200.15,50307.00,50178.45,50194.80,0,NaN
2024-11-21 09:25:00,50193.35,50245.00,50162.30,50186.95,0,NaN
2024-11-21 09:30:00,50190.65,50207.00,49950.45,49953.10,0,NaN
2024-11-21 09:35:00,49957.10,49970.05,49821.95,49822.65,0,49988.376540
...,...,...,...,...,...,...
2024-11-22 15:05:00,51148.80,51178.85,51131.25,51161.25,0,51164.876565
2024-11-22 15:10:00,51160.85,51174.10,51122.95,51141.10,0,51156.951043
2024-11-22 15:15:00,51141.00,51141.95,51043.55,51096.40,0,51136.767362


In [24]:
# on message function 
def onmessage(message):
    #symb = message['symbol']
    #ltp = message['ltp']
    #print(message)
    t = time.localtime()
    cmin = time.strftime("%M", t)
    csec = time.strftime("%S", t)
    global pos, stoploss, exit, strike, target, flag
    if (int(cmin) % 5 == 0 and int(csec) < 3):
        print("5 ema data updated")
        getdata()
        time.sleep(1)
        if (pos == 0):
            flag = 0
    symb = "NSE:NIFTYBANK-INDEX"
    ema = emadata['ema'].iloc[-2]
    l =  emadata['low'].iloc[-2]
    print(f"{message} | low {l} |ema {ema} | ")
    if (emadata['close'].iloc[-2] > emadata['ema'].iloc[-2] 
        and emadata['high'].iloc[-2] > emadata['ema'].iloc[-2]
        and emadata['open'].iloc[-2] > emadata['ema'].iloc[-2]
        and emadata['low'].iloc[-2] > emadata['ema'].iloc[-2]
        and message['ltp'] < emadata['low'].iloc[-2]):
        ltp = message['ltp']
        sp = int(round(ltp,-2))
        if (pos==0 and flag==0):
            strike = "NSE:BANKNIFTY24NOV" + str(sp) + "PE"
            data = {
                "symbol": str(strike),
                "qty":15,
                "type":2,
                "side":1,
                "productType":"MARGIN",
                "limitPrice":0,
                "stopPrice":0,
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":False,
                }
            print(f"entry {strike}")
            #response = fyers.place_order(data=data)
            mongoObj.write(data)
            pos = flag = 1
            entry = message['ltp']
            stoploss = emadata['high'].iloc[-2]
            target = message['ltp'] - ((emadata['high'].iloc[-2] - emadata['low'].iloc[-2])*3)
            print(response)
    if (pos==1 and message['ltp'] >= stoploss):
            data = {
            "symbol": str(strike),
            "qty":15,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
            print("stop loss")
            #response = fyers.place_order(data=data)
            mongoObj.write(data)
            print(response)
            pos = 0
            stoploss = 0
            entry = 0
            target = 0
            time.sleep(1)
            data_type = "SymbolUpdate"
            symbols_to_unsubscribe = ['NSE:NIFTYBANK-INDEX']
            fyersdata.unsubscribe(symbols=symbols_to_unsubscribe, data_type=data_type)
    if (pos==1 and message['ltp'] <= target):
            data = {
            "symbol": str(strike),
            "qty":15,
            "type":2,
            "side":-1,
            "productType":"MARGIN",
            "limitPrice":0,
            "stopPrice":0,
            "validity":"DAY",
            "disclosedQty":0,
            "offlineOrder":False,
            }
            print("traget ")
            #response = fyers.place_order(data=data)
            mongoObj.write(data)
            print(response)
            pos = 0
            stoploss = 0
            entry = 0
            target = 0
            time.sleep(1)
            data_type = "SymbolUpdate"
            symbols_to_unsubscribe = ['NSE:NIFTYBANK-INDEX']
            fyersdata.unsubscribe(symbols=symbols_to_unsubscribe, data_type=data_type)
            print(f"symbol {symb} and ltp {ltp} | low {l} |ema {ema} | ")
            print(f"success {message}")
def onerror(message):
    print("Error:", message)
def onclose(message):
    print("Connection closed:", message)
def onopen():
    data_type = "SymbolUpdate"
    symbols = ['NSE:NIFTYBANK-INDEX']
    fyersdata.subscribe(symbols=symbols, data_type=data_type)
    fyersdata.keep_running()


In [25]:
# Replace the sample access token with your actual access token obtained from Fyers
#access_token = client_id + ":" + tk
# Create a FyersDataSocket instance with the provided parameters
fyersdata = data_ws.FyersDataSocket(
     access_token=access_token,       # Access token in the format "appid:accesstoken"
     log_path="",                     # Path to save logs. Leave empty to auto-create logs in the current directory.
     litemode=True,                  # Lite mode disabled. Set to True if you want a lite response.
     write_to_file=False,              # Save response in a log file instead of printing it.
     reconnect=True,                  # Enable auto-reconnection to WebSocket on disconnection.
     on_connect=onopen,               # Callback function to subscribe to data upon connection.
     on_close=onclose,                # Callback function to handle WebSocket connection close events.
     on_error=onerror,                # Callback function to handle WebSocket errors.
     on_message=onmessage             # Callback function to handle incoming messages from the WebSocket.
 )
# Establish a connection to the Fyers WebSocket
fyersdata.connect()


2024-11-22 18:15:42,435 INFO:
Websocket connected



{'type': 'cn', 'code': 200, 'message': 'Authentication done', 's': 'ok'} | low 51047.95 |ema 51137.01157486645 | 
{'type': 'lit', 'code': 200, 'message': 'Lite Mode On', 's': 'ok'} | low 51047.95 |ema 51137.01157486645 | 


2024-11-22 18:15:44,263 DEBUG:
Starting new HTTPS connection (1): api-t1.fyers.in:443

2024-11-22 18:15:44,425 DEBUG:
https://api-t1.fyers.in:443 "POST /data/symbol-token HTTP/1.1" 200 None



{'type': 'sub', 'code': 200, 'message': 'Subscribed', 's': 'ok'} | low 51047.95 |ema 51137.01157486645 | 
{'ltp': 51135.4, 'symbol': 'NSE:NIFTYBANK-INDEX', 'type': 'if'} | low 51047.95 |ema 51137.01157486645 | 
